In [1]:
import pandas as pd
import numpy as np
import ast
from datetime import datetime

načtení původní tabulky

In [2]:
trafficDelaysDf = pd.read_csv('C:/Users/verav/Documents/Projekt/Python_projekt/plynulostVSE.csv')

In [3]:
trafficDf = trafficDelaysDf.loc[:,['properties.uuid','properties.street']]
trafficDf.columns = trafficDf.columns.str.replace('properties.','')

In [4]:
trafficDf.head()

,uuid,street
0,1363839630,» Centrum / Bohunice
1,1388730276,NaN
2,1451737922,U Dráhy
3,1388985166,Na Hrázi
4,1440208034,NaN


In [7]:
streets2Df = pd.read_csv('missing_streets2.csv', encoding = 'UTF-8') #načtení prvního souboru doplněných ulic
streets2Df = streets2Df.drop(['Unnamed: 0', 'Unnamed: 0.1','missingStreet','firstCoord'], axis = 1)
streets2Df.head()

,uuid,streets2
0,1388730276,Labská
1,1440208034,Okrouhlá
2,1370182069,Labská
3,1415306095,"Starý Lískovec,okres Brno-město"
4,1415188035,Okrouhlá


zbavení se nedoplněných ulic (městská část,okres) v Streets2

In [8]:
def nullMissing(vstup):#funkce na náhradu špatného názvu prázdným řetězcem
    if 'okres' in vstup:
        new = vstup.replace(vstup,"")
    else:
        new = vstup
    return new

In [9]:
streets2Df['streets2'].apply(nullMissing)[0:5]

0      Labská
1    Okrouhlá
2      Labská
3            
4    Okrouhlá
Name: streets2, dtype: object

In [10]:
streets2Df['streets2'] = streets2Df['streets2'].apply(nullMissing)

In [11]:
streetsMerge1Df = trafficDf.merge(streets2Df,how ='left', on = 'uuid') #propojení tabulek podle uuid

In [12]:
streetsMerge1Df.head(10)

,uuid,street,streets2
0,1363839630,» Centrum / Bohunice,NaN
1,1388730276,NaN,Labská
2,1451737922,U Dráhy,NaN
3,1388985166,Na Hrázi,NaN
4,1440208034,NaN,Okrouhlá
5,1370182069,NaN,Labská
6,1415306095,NaN,
7,1363840440,» Centrum / Bohunice,NaN
8,1415188035,NaN,Okrouhlá
9,1531577943,NaN,Jihlavská


In [14]:
streetsMerge1Df['streets2'] = streetsMerge1Df['streets2'].fillna("")
streetsMerge1Df['street'] = streetsMerge1Df['street'].fillna("")

In [15]:
streetsMerge1Df.head(10)

,uuid,street,streets2
0,1363839630,» Centrum / Bohunice,
1,1388730276,,Labská
2,1451737922,U Dráhy,
3,1388985166,Na Hrázi,
4,1440208034,,Okrouhlá
5,1370182069,,Labská
6,1415306095,,
7,1363840440,» Centrum / Bohunice,
8,1415188035,,Okrouhlá
9,1531577943,,Jihlavská


doplnění názvů do sloupce street ze sloupce streets2

In [16]:
street = []
for i in streetsMerge1Df.index:
    if len(streetsMerge1Df['street'][i]) == 0:
        street.append(streetsMerge1Df['streets2'][i])
    else:
        street.append(streetsMerge1Df['street'][i])

In [17]:
streetsMerge1Df['street'] = street
streetsMerge1Df.head()

,uuid,street,streets2
0,1363839630,» Centrum / Bohunice,
1,1388730276,Labská,Labská
2,1451737922,U Dráhy,
3,1388985166,Na Hrázi,
4,1440208034,Okrouhlá,Okrouhlá


kontrola počtu chybějících názvů ulic

In [18]:
pocetMiss1 = []
for i in streetsMerge1Df.index:
    if len(streetsMerge1Df['street'][i]) == 0:
        pocetMiss1.append(streetsMerge1Df['street'][i])

In [19]:
len(pocetMiss1)

885

2.kolo doplňování názvů (z poslední souřadnice)

In [20]:
streets3Df = pd.read_csv('missing_streets3.csv', encoding = 'UTF-8')
streets3Df = streets3Df[['uuid','street3']]
streets3Df.head()

,uuid,street3
0,1415306095,Okrouhlá
1,1337977851,Charbulova
2,1415181365,"Moravany u Brna,okres Brno-venkov"
3,1531488523,Bubeníčkova
4,1440329274,"Moravany u Brna,okres Brno-venkov"
5,1364049280,"Moravany u Brna,okres Brno-venkov"
6,1531481363,"Moravany u Brna,okres Brno-venkov"
7,1370400299,Mrštíkova
8,1411773276,"Nebovidy u Brna,okres Brno-venkov"
9,1403581057,Brněnská


In [22]:
streetsMerge2Df = streetsMerge1Df.merge(streets3Df,how ='left', on = 'uuid') #propojení tabulek podle uuid

In [23]:
streetsMerge2Df.head()

,uuid,street,streets2,street3
0,1363839630,» Centrum / Bohunice,,NaN
1,1388730276,Labská,Labská,NaN
2,1451737922,U Dráhy,,NaN
3,1388985166,Na Hrázi,,NaN
4,1440208034,Okrouhlá,Okrouhlá,NaN
5,1370182069,Labská,Labská,NaN
6,1415306095,,,Okrouhlá
7,1363840440,» Centrum / Bohunice,,NaN
8,1415188035,Okrouhlá,Okrouhlá,NaN
9,1531577943,Jihlavská,Jihlavská,NaN


In [24]:
streetsMerge2Df['street3']= streetsMerge2Df['street3'].fillna("")

In [25]:
streetsMerge2Df

,uuid,street,streets2,street3
0,1363839630,» Centrum / Bohunice,,
1,1388730276,Labská,Labská,
2,1451737922,U Dráhy,,
3,1388985166,Na Hrázi,,
4,1440208034,Okrouhlá,Okrouhlá,
...,...,...,...,...
116704,1503042209,Lesnická,,
116705,435163611,Oblá,,
116706,649666312,Koliště,,
116707,649898012,374,,


In [31]:
streetList2 = []
for i in streetsMerge2Df.index:
    if len(streetsMerge2Df['street'][i]) == 0:
        new = streetsMerge2Df['street3'][i]
        streetList2.append(new)
    else:
        streetList2.append(streetsMerge2Df['street'][i])

In [40]:
streetsMerge2Df['street'] = streetList2
streetsMerge2Df.head()

,uuid,street,streets2,street3
0,1363839630,» Centrum / Bohunice,,
1,1388730276,Labská,Labská,
2,1451737922,U Dráhy,,
3,1388985166,Na Hrázi,,
4,1440208034,Okrouhlá,Okrouhlá,
5,1370182069,Labská,Labská,
6,1415306095,Okrouhlá,,Okrouhlá
7,1363840440,» Centrum / Bohunice,,
8,1415188035,Okrouhlá,Okrouhlá,
9,1531577943,Jihlavská,Jihlavská,


In [46]:
streetsMerge2Df['street']= streetsMerge2Df['street'].fillna("") #převede automaticky vše na string

In [73]:
chybi = []

In [74]:
for i in streetsMerge2Df.index:
    if 'okres' in streetsMerge2Df['street'][i]:
        chybi.append(streetsMerge2Df['street'][i])

In [75]:
len(chybi)

462

In [76]:
streetsMerge2Df['street'] = streetsMerge2Df['street'].str.strip()

z předchozího postupu zbývá nedoplněných ulic cca 462, doplněno alespoň městskou částí a názvem obce...

In [79]:
finalDfStreet = streetsMerge2Df[['uuid','street']]
len(finalDfStreet)

116709

In [ ]:
finalDfStreet.to_csv('finalDfStreet.csv', encoding = 'utf-8', index = False)